# You Do Session 2
- Importing Libraries and the data

In [8]:
import numpy as np
import pandas as pd


In [9]:
# uploading data and making it rating matrix 
df = pd.read_csv('https://files.grouplens.org/datasets/movielens/ml-100k/u.data', delimiter=r'\t',
names=['user_id', 'item_id', 'rating', 'timestamp'])
R = df.pivot(index='user_id', columns='item_id', values='rating').values
R

array([[ 5.,  3.,  4., ..., nan, nan, nan],
       [ 4., nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [ 5., nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan,  5., nan, ..., nan, nan, nan]])

In [10]:
# look for the shape of the matrix
R.shape

(943, 1682)

In [11]:
# we need nan values to fill the matrix, so we need to mask the data 
# find indexes of non-zero elements
irow, jcol = np.where(~np.isnan(R))

## 1 ) Non-regularized version

### Let's create our model, our loss functions

- Our model will be bias based as declared in the homework description. 

- $$ \hat{r_{ij}} = b_i^{user} + b_j^{item} $$

- As we declared above, we should optimize our b values over non-zero values. Let's call this subset S and define it as follows:

- $$  S = \left \{ (i,j) : r_{ij}\:is\: observed \right \} $$ 

- Then our loss function will look like this 

- $$ loss = \frac{1}{2} * \sum_{(i,j) \in S }( r_{ij} - \hat{r_{ij}})^{2}  $$


In [27]:
# splitting data into training and test sets
def split_data(R,irow,jcol):
    idx = np.random.choice(np.arange(100_000), 10000, replace=False)
    test_irow = irow[idx]
    test_jcol = jcol[idx]
    R_copy = R.copy()
    R_copy[test_irow, test_jcol] = np.nan
    return R_copy, test_irow, test_jcol


In [28]:
# define loss function
def loss(b_user, b_item, R, irow, jcol):
    loss = 0
    for i, j in zip(irow, jcol):
        if np.isnan(R[i, j]):
            continue
        loss += (R[i, j] - b_user[i] - b_item[j]) ** 2 * 0.5
    return loss


In [29]:
# define gradient of loss function
def gradient(b_user, b_item, R, irow, jcol):
    b_user_grad = np.zeros(b_user.shape)
    b_item_grad = np.zeros(b_item.shape)
    for i, j in zip(irow, jcol):
        if np.isnan(R[i, j]):
            continue
        b_user_grad[i] += (R[i, j] - b_user[i] - b_item[j])
        b_item_grad[j] += (R[i, j] - b_user[i] - b_item[j])

    return b_user_grad, b_item_grad


In [30]:
# define gradient descent function
def gradient_descent(R, b_user, b_item, irow, jcol, lr, epochs):
    for epoch in range(epochs):
        b_user_grad, b_item_grad = gradient(b_user, b_item, R, irow, jcol)
        prev_b_user = b_user.copy()
        prev_b_item = b_item.copy()
        b_user += lr * b_user_grad
        b_item += lr * b_item_grad
        if epoch % 10 == 0:
            print('loss:', loss(b_user, b_item, R, irow, jcol))
        # very early stopping
        if np.linalg.norm(b_user - prev_b_user) < 1e-2 and np.linalg.norm(b_item - prev_b_item) < 1e-2:
            break

    return b_user, b_item

In [31]:
# calculate RMSE
def rmse(R, b_user, b_item, irow, jcol):
    return np.sqrt(2 * loss(b_user, b_item, R, irow, jcol) / len(irow))

In [33]:
# split data into training and test sets
R_copy, test_irow, test_jcol = split_data(R,irow,jcol)


# initialize biases
b_user = np.random.randn(943)
b_item = np.random.randn(1682)


# run gradient descent
b_user, b_item = gradient_descent(R_copy, b_user, b_item, irow, jcol, 0.0005, 1000)

# calculate RMSE on test set
print('Non-Regularized RMSE for test set:' , rmse(R, b_user, b_item, test_irow, test_jcol))

loss: 512701.9190788698
loss: 113572.47119923071
loss: 76150.66835639812
loss: 62733.18443845988
loss: 55675.78633463886
loss: 51327.840741421125
loss: 48404.009012990886
loss: 46324.54118858998
loss: 44785.94704069288
loss: 43613.0230151942
loss: 42697.4802643668
loss: 41968.922713540596
loss: 41379.7512497769
loss: 40896.73655421998
loss: 40496.0532920679
loss: 40160.2256277335
loss: 39876.180843786344
loss: 39633.97130561652
loss: 39425.91311176596
loss: 39245.9920486845
loss: 39089.44538291593
loss: 38952.461949876146
loss: 38831.96346042623
loss: 38725.44260807699
loss: 38630.841576217856
loss: 38546.459727342946
loss: 38470.882671411746
loss: 38402.92720118798
loss: 38341.59814444658
loss: 38286.05426447703
loss: 38235.58110003209
loss: 38189.56917662066
loss: 38147.49641083617
loss: 38108.913813627136
loss: 38073.43380788024
loss: 38040.72063176197
loss: 38010.48241645246
loss: 37982.46461590825
loss: 37956.44453420481
loss: 37932.226748467685
loss: 37909.63926601646
loss: 37888

## 2 ) Regularized version

### Let's create  our loss functions


- Our loss function will look like this when we add regularized terms

 $$ regularized \: loss = \frac{1}{2} * \sum_{(i,j) \in S }( r_{ij} - \hat{r_{ij}})^{2} + \frac{\lambda}{2} (\sum_{i =1}^{m} (b_i^{user})^2 + \sum_{j =1}^{n} (b_j^{item})^2  )   $$

In [34]:
# regularized loss function
def loss_reg(b_user, b_item, R, irow, jcol, lmbda):
    loss = 0
    for i, j in zip(irow, jcol):
        if np.isnan(R[i, j]):
            continue
        loss += (R[i, j] - b_user[i] - b_item[j]) ** 2 * 0.5 + lmbda/2 * (b_user[i] ** 2 + b_item[j] ** 2)
    return loss

In [35]:
# regularized gradient of loss function
def gradient_reg(b_user, b_item, R, irow, jcol, lmbda):
    b_user_grad = np.zeros(b_user.shape)
    b_item_grad = np.zeros(b_item.shape)
    for i, j in zip(irow, jcol):
        if np.isnan(R[i, j]):
            continue
        b_user_grad[i] += (R[i, j] - b_user[i] - b_item[j]) - lmbda * b_user[i]
        b_item_grad[j] += (R[i, j] - b_user[i] - b_item[j]) - lmbda * b_item[j]

    return b_user_grad, b_item_grad

In [36]:
# regularized gradient descent function
def gradient_descent_reg(R, b_user, b_item, irow, jcol, lmbda, lr, epochs):
    for epoch in range(epochs):
        b_user_grad, b_item_grad = gradient_reg(b_user, b_item, R, irow, jcol, lmbda)
        prev_b_user = b_user.copy()
        prev_b_item = b_item.copy()
        b_user += lr * b_user_grad
        b_item += lr * b_item_grad
        if epoch % 10 == 0:
            print('loss:', loss_reg(b_user, b_item, R, irow, jcol, lmbda))
        # very early stopping
        if np.linalg.norm(b_user - prev_b_user) < 1e-2 and np.linalg.norm(b_item - prev_b_item) < 1e-2:
            break

    return b_user, b_item

In [37]:
# cross-validation to find best lambda and save best model on test set
b_user = np.random.randn(943)
b_item = np.random.randn(1682)
best_rmse = float('inf')
best_lmbda = 0
for lmbda in [0, 0.001, 0.01, 0.1, 1]:
    b_user, b_item = gradient_descent_reg(R_copy, b_user, b_item, irow, jcol, lmbda, 0.0005, 250)
    rmse_val = rmse(R, b_user, b_item, test_irow, test_jcol)
    if rmse_val < best_rmse:
        best_rmse = rmse_val
        best_lmbda = lmbda


loss: 524315.7959500112
loss: 113947.7050960819
loss: 76524.86772735107
loss: 63077.09052528317
loss: 55949.362315047234
loss: 51533.62093178076
loss: 48553.646871440695
loss: 46429.943941172256
loss: 44857.31565460672
loss: 43658.593950065624
loss: 42723.71141983058
loss: 41980.78891420842
loss: 41381.04163373995
loss: 40890.316048047825
loss: 40484.07780990949
loss: 40144.31247894359
loss: 39857.540473206005
loss: 39613.507629853746
loss: 39404.29926526799
loss: 39223.727308991954
loss: 39066.89789253693
loss: 38929.900806005666
loss: 38809.58287570565
loss: 38703.38016141
loss: 38609.19205025042
loss: 38824.97985316854
loss: 38750.10711924921
loss: 38682.933007689964
loss: 38622.37048189101
loss: 38567.56972348508
loss: 38517.81225840944
loss: 38472.48633457268
loss: 38431.06873092202
loss: 38393.110148774176
loss: 38358.22331006789
loss: 38326.07319288693
loss: 38296.36897811471
loss: 38268.85737675034
loss: 38243.31707778904
loss: 38219.55411018914
loss: 38197.397953964624
loss: 3

In [38]:
# run regularized gradient descent with best lambda
b_user, b_item = gradient_descent_reg(R_copy, b_user, b_item, irow, jcol, best_lmbda, 0.0005, 1000)



# function for regularized RMSE on test set with best model
def rmse_reg(R, b_user, b_item, irow, jcol, lmbda):
    return np.sqrt(2 * loss_reg(b_user, b_item, R, irow, jcol, lmbda) / len(irow))


# calculate RMSE on test set
print('Regularized RMSE:' , rmse_reg(R, b_user, b_item, test_irow, test_jcol, best_lmbda))

loss: 86245.40055533985
loss: 46587.628422242946
loss: 43551.26541806876
loss: 42495.23640979734
loss: 41931.92899519717
loss: 41578.96255717097
loss: 41338.00367333885
loss: 41164.42544389834
loss: 41034.617953019304
loss: 40934.787368595986
loss: 40856.3030272344
loss: 40793.486611623906
loss: 40742.45443285412
loss: 40700.465963644994
loss: 40665.536604673376
loss: 40636.19734334324
loss: 40611.34020903478
loss: 40590.1158781658
loss: 40571.864048363124
loss: 40556.06499931369
loss: 40542.30519956483
loss: 40530.252436432325
loss: 40519.63753476415
loss: 40510.24071922339
loss: 40501.881304613955
loss: 40494.409808304845
loss: 40487.70185031717
loss: 40481.65338972584
loss: 40476.17697174758
loss: 40471.1987473216
loss: 40466.65608892336
loss: 40462.49567062005
loss: 40458.67191251838
loss: 40455.14571338737
loss: 40451.88341270176
loss: 40448.855936478525
loss: 40446.03809123324
loss: 40443.407977903
Regularized RMSE: 0.9804548430366312
